In [ ]:
###Calculate forest - Non forest % for landsat 7

In [ ]:
import os
import re
import numpy as np
import rasterio
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import scipy.ndimage as ndimage

size = 256
model = load_model('forest_detection_model_vgg16_unet_21042025i2.h5')

def scaleStd(x):
    return (x - (np.nanmean(x) - np.nanstd(x) * 2)) / (
        (np.nanmean(x) + np.nanstd(x) * 2) - (np.nanmean(x) - np.nanstd(x) * 2))

def preprocess_input(image):
    image = image / 255.0
    return image

def fill_gaps_with_interpolation(band):
    if np.ma.isMaskedArray(band):
        band = band.filled(np.nan)
    mask = np.isnan(band)
    filled_band = ndimage.generic_filter(band, np.nanmean, size=3)
    band[mask] = filled_band[mask]
    return band

def calculate_deforestation_per_year(image_paths, tile_size=size):
    deforestation_percentages = []
    forest_percentages = []

    for image_path in image_paths:
        tiles = preprocess_image(image_path, tile_size)
        predictions = [model.predict(np.expand_dims(tile, axis=0)) for tile in tiles]

        deforestation_percentages_i = []
        for prediction in predictions:
            sum_prediction = np.sum(prediction, axis=-1)
            non_zero_mask = sum_prediction != 0
            deforestation_percentage = np.zeros_like(sum_prediction)
            deforestation_percentage[non_zero_mask] = (
                sum_prediction[non_zero_mask] / sum_prediction[non_zero_mask].max()) * 100
            deforestation_percentages_i.append(np.nanmean(deforestation_percentage))

        avg_deforestation = np.nanmean(deforestation_percentages_i)
        deforestation_percentages.append(avg_deforestation)
        forest_percentages.append(100 - avg_deforestation)

    return deforestation_percentages, forest_percentages

def visualize_deforestation(image_paths):
    deforestation_percentages, forest_percentages = calculate_deforestation_per_year(image_paths)

    for i, image_path in enumerate(image_paths):
        year = extract_year_from_filename(os.path.basename(image_path))
        print(f"Non-Forest Percentage - {year}: {deforestation_percentages[i]:.2f}%")
        print(f"Remaining Forest Percentage - {year}: {forest_percentages[i]:.2f}%")

def extract_year_from_filename(filename):
    match = re.search(r'\d{4}', filename)
    return int(match.group()) if match else None

def preprocess_image(image_path, tile_size=size):
    with rasterio.open(image_path) as src:
        red = fill_gaps_with_interpolation(src.read(1, masked=True))
        green = fill_gaps_with_interpolation(src.read(2, masked=True))
        blue = fill_gaps_with_interpolation(src.read(3, masked=True))

    r_std = scaleStd(red)
    g_std = scaleStd(green)
    b_std = scaleStd(blue)

    rgb_std = np.dstack((r_std, g_std, b_std))
    rgb_std = np.clip(rgb_std, 0, 1)
    rgb_std = (rgb_std * 255).astype(np.uint8)

    img_height, img_width, _ = rgb_std.shape

    tiles = []
    for y in range(0, img_height, tile_size):
        for x in range(0, img_width, tile_size):
            tile = rgb_std[y:y + tile_size, x:x + tile_size, :]
            tiles.append(tile)

    max_width = max(tile.shape[1] for tile in tiles)
    max_height = max(tile.shape[0] for tile in tiles)
    padded_tiles = [
        np.pad(tile, ((0, max_height - tile.shape[0]), (0, max_width - tile.shape[1]), (0, 0)), 'constant')
        for tile in tiles
    ]
    preprocessed_tiles = [preprocess_input(img_to_array(tile)) for tile in padded_tiles]

    return preprocessed_tiles

# Directory of normalized Landsat images (one per year)
image_directory = "testimage/sund/landsat_7_interpolation"
image_paths = [os.path.join(image_directory, f) for f in os.listdir(image_directory) if f.endswith(".tif")]

# Run deforestation analysis
visualize_deforestation(image_paths)


In [ ]:
###Calculate forest - Non forest % for landsat 8

In [ ]:
import tensorflow as tf
size = 256
# Load the pre-trained U-Net model
model = load_model('forest_detection_model_vgg16_unet_21042025i2.h5')

def scaleStd(x):
    return (x - (np.nanmean(x) - np.nanstd(x) * 2)) / ((np.nanmean(x) + np.nanstd(x) * 2) - (np.nanmean(x) - np.nanstd(x) * 2))

def preprocess_input(image):
    # Assuming pixel values are in the range [0, 255]
    image = image / 255.0
    return image

def calculate_deforestation_per_year(image_paths, tile_size=size):
    num_years = len(image_paths)
    
    deforestation_percentages = []
    forest_percentages = []
    
    for i in range(num_years):
        # Preprocess the image and split into tiles
        tiles = preprocess_image(image_paths[i], tile_size)

        # Predict using the trained model for all tiles
        predictions = [model.predict(np.expand_dims(tile, axis=0)) for tile in tiles]

        # Calculate the percentage of non-forest for each tile
        deforestation_percentages_i = []
        for prediction in predictions:
            sum_prediction = np.sum(prediction, axis=-1)
            
            # Calculate the percentage of non-forest (deforestation)
            non_zero_mask = sum_prediction != 0
            deforestation_percentage = np.zeros_like(sum_prediction)
            deforestation_percentage[non_zero_mask] = (
                sum_prediction[non_zero_mask] / sum_prediction[non_zero_mask].max()
            ) * 100

            deforestation_percentages_i.append(np.nanmean(deforestation_percentage))

        # Calculate overall deforestation and forest percentage for the year
        avg_deforestation = np.nanmean(deforestation_percentages_i)
        deforestation_percentages.append(avg_deforestation)
        forest_percentages.append(100 - avg_deforestation)

    return deforestation_percentages, forest_percentages


def visualize_deforestation(image_paths):
    deforestation_percentages, forest_percentages = calculate_deforestation_per_year(image_paths)

    for i, image_path in enumerate(image_paths):
        year = extract_year_from_filename(os.path.basename(image_path))

        deforestation_text = f"Non-Forest Percentage - {year}: {deforestation_percentages[i]:.2f}%"
        forest_text = f"Remaining Forest Percentage - {year}: {forest_percentages[i]:.2f}%"
        
        print(deforestation_text)
        print(forest_text)


def extract_year_from_filename(filename):
    # Extract the last 4 digits from the filename
    match = re.search(r'\d{4}', filename)
    if match:
        return int(match.group())
    else:
        return None
    
def preprocess_image(image_path, tile_size=size):
    # Load the image using rasterio for specific band access
    with rasterio.open(image_path) as src:
        red = src.read(1, masked=True)
        green = src.read(2, masked=True)
        blue = src.read(3, masked=True)

    # Normalize bands
    r_std = scaleStd(red)
    g_std = scaleStd(green)
    b_std = scaleStd(blue)

    # Stack normalized bands into RGB image
    rgb_std = np.dstack((r_std, g_std, b_std))

    # Convert to float32 and scale to 0–255
    rgb_std = np.clip(rgb_std, 0, 1)  # Make sure values are within range
    rgb_std = (rgb_std * 255).astype(np.uint8)

    img_height, img_width, _ = rgb_std.shape

    # Split into tiles
    tiles = []
    for y in range(0, img_height, tile_size):
        for x in range(0, img_width, tile_size):
            tile = rgb_std[y:y+tile_size, x:x+tile_size, :]
            tiles.append(tile)

    # Pad tiles with zeros to match maximum height/width
    max_width = max(tile.shape[1] for tile in tiles)
    max_height = max(tile.shape[0] for tile in tiles)
    padded_tiles = [
        np.pad(tile, ((0, max_height - tile.shape[0]), (0, max_width - tile.shape[1]), (0, 0)), 'constant')
        for tile in tiles
    ]

    # Preprocess for model input
    preprocessed_tiles = [preprocess_input(img_to_array(tile)) for tile in padded_tiles]

    return preprocessed_tiles


# Provide the directory containing satellite images for different years
image_directory = "testimage/sund/landsat_8_normalized/2013-2023"
image_paths = [os.path.join(image_directory, filename) for filename in os.listdir(image_directory) if filename.endswith(".tif")]

# Visualize deforestation on the original images without showing the map
visualize_deforestation(image_paths)